#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [24]:
collection.delete_many({})

In [1]:
import pandas as pd
import pymongo

In [2]:
client = pymongo.MongoClient("mongo")
database = client['exercices']
collection = database['kickstarter']

In [3]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

/root/.local/share/virtualenvs/code-AFz48Hjb/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (6,8,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [4]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

In [5]:
df_ks['goal']=df_ks['goal'].apply(lambda x: pd.to_numeric(x, errors='coerce'))
df_ks['pledged']=df_ks['pledged'].apply(lambda x: pd.to_numeric(x, errors='coerce'))
df_ks['backers']=df_ks['backers'].apply(lambda x: pd.to_numeric(x, errors='coerce'))
df_ks['usd pledged']=df_ks['usd pledged'].apply(lambda x: pd.to_numeric(x, errors='coerce',downcast='float'))

In [6]:
df_ks.shape

(150000, 14)

## Question 0

### Netoyer les données

In [7]:
df_ks.dropna(axis=0,inplace=True)

In [8]:
df_sample=df_ks.sample(frac=1/4,axis=0)
df_sample.shape

(37129, 14)

In [9]:
df_ks.dtypes

ID                    int64
name                 object
category             object
main_category        object
currency             object
deadline             object
goal                float64
launched             object
pledged             float64
state                object
backers             float64
country              object
usd pledged         float64
usd_pledged_real    float64
dtype: object

import datetime
for i in range(len(df_ks)):
    datetime.datetime.strptime(df_ks['deadline'].iloc[i], '%Y%m%d') 


In [10]:
df_ks['launched']=pd.to_datetime(df_ks['launched'])

In [11]:
df_ks['deadline']=pd.to_datetime(df_ks['deadline'])

In [12]:
df_ks.dtypes

ID                           int64
name                        object
category                    object
main_category               object
currency                    object
deadline            datetime64[ns]
goal                       float64
launched            datetime64[ns]
pledged                    float64
state                       object
backers                    float64
country                     object
usd pledged                float64
usd_pledged_real           float64
dtype: object

### Importer les données

In [13]:
def slice_generator(df, chunk_size=10):
    current_row = 0
    total_rows = df.shape[0]
    while current_row < total_rows:
        yield df[current_row:current_row + chunk_size]
        current_row += chunk_size


In [14]:
for df_chunk in slice_generator(df_sample):
    records = df_chunk.to_dict(orient='records')
    collection.insert_many(records)


In [15]:
collection.find_one()

{'_id': ObjectId('5fccbfdc247348e355264517'),
 'ID': 1845007191,
 'name': 'Going To The Open, The Story of a Pool Underdog (Canceled)',
 'category': 'Nonfiction',
 'main_category': 'Publishing',
 'currency': 'USD',
 'deadline': '2014-02-03',
 'goal': 3000.0,
 'launched': '2013-12-05 23:09:50',
 'pledged': 0.0,
 'state': 'canceled',
 'backers': 0.0,
 'country': 'US',
 'usd pledged': 0.0,
 'usd_pledged_real': 0.0}

## Question 1  

cur=collection.find().sort('backers',-1).limit(5)
list(cur)

In [16]:
cur = collection.aggregate([{"$group" : {"_id" : "$ID", "averagebackers" : {"$avg" : "$backers"}}},{"$sort":{"averagebackers":-1}},{"$limit":5}])
list(cur)

[{'_id': 1386523707, 'averagebackers': 154926.0},
 {'_id': 2103598555, 'averagebackers': 66673.0},
 {'_id': 1033978702, 'averagebackers': 63416.0},
 {'_id': 342886736, 'averagebackers': 62642.0},
 {'_id': 951470114, 'averagebackers': 28138.0}]

## Question 2

In [17]:
cur=collection.find({"state":"successful"})

len(list(cur))

13351

In [18]:
cur=collection.aggregate([{"$group" : {"_id" : "$state", "totalnumber" : {"$sum" : 1}}}])
list(cur)

[{'_id': 'canceled', 'totalnumber': 3838},
 {'_id': 'successful', 'totalnumber': 13351},
 {'_id': 'suspended', 'totalnumber': 193},
 {'_id': 'live', 'totalnumber': 279},
 {'_id': 'failed', 'totalnumber': 19468}]

## Question 3

In [19]:
cur=collection.aggregate([{"$group":{"_id":"$category","total":{"$sum":1}}}])
list(cur)

[{'_id': 'Festivals', 'total': 77},
 {'_id': 'Theater', 'total': 711},
 {'_id': 'Art Books', 'total': 297},
 {'_id': 'Civic Design', 'total': 33},
 {'_id': 'Classical Music', 'total': 253},
 {'_id': 'Woodworking', 'total': 122},
 {'_id': 'Webcomics', 'total': 69},
 {'_id': 'Family', 'total': 33},
 {'_id': 'Accessories', 'total': 311},
 {'_id': 'Product Design', 'total': 2185},
 {'_id': 'Experimental', 'total': 83},
 {'_id': 'People', 'total': 113},
 {'_id': 'Apparel', 'total': 707},
 {'_id': 'Flight', 'total': 45},
 {'_id': 'Art', 'total': 846},
 {'_id': 'Ready-to-wear', 'total': 95},
 {'_id': 'Video Art', 'total': 14},
 {'_id': 'Music', 'total': 1337},
 {'_id': 'Photography', 'total': 560},
 {'_id': 'Nonfiction', 'total': 863},
 {'_id': 'Knitting', 'total': 15},
 {'_id': 'Digital Art', 'total': 116},
 {'_id': 'Academic', 'total': 98},
 {'_id': 'Gadgets', 'total': 311},
 {'_id': 'Young Adult', 'total': 76},
 {'_id': 'Horror', 'total': 133},
 {'_id': 'Residencies', 'total': 7},
 {'_id':

## Question 4

collection.find({"launched":{ "$lte" :  "2015-12-31"}}).count(True)


In [20]:
cur=collection.find({"launched":{ "$lte" :  "2015-12-31"}})

len(list(cur))

26313

import datetime
start=datetime.datetime(2016,1,1)

type(start)
print(start)

cur=collection.find({"launched":{"$lt":start}})
list(cur)

cur=collection.find({"deadline" : { "$gt" : start}})
len(list(cur))

## Question 5

In [21]:
cur=collection.find({"pledged":{ "$gte" :  200000}})
list(cur)

[{'_id': ObjectId('5fccbfdc247348e35526453e'),
  'ID': 879028973,
  'name': "RiffTrax Live '17: Samurai Cop, Beach Party & Mystery Title!",
  'category': 'Film & Video',
  'main_category': 'Film & Video',
  'currency': 'USD',
  'deadline': '2017-03-26',
  'goal': 250000.0,
  'launched': '2017-02-21 20:59:19',
  'pledged': 430620.15,
  'state': 'successful',
  'backers': 8602.0,
  'country': 'US',
  'usd pledged': 83630.5234375,
  'usd_pledged_real': 430620.15},
 {'_id': ObjectId('5fccbfdc247348e3552645a1'),
  'ID': 909248984,
  'name': 'Redux COURG - Hybrid Watches with Missions to Tackle',
  'category': 'Product Design',
  'main_category': 'Design',
  'currency': 'USD',
  'deadline': '2015-08-20',
  'goal': 30000.0,
  'launched': '2015-07-21 19:01:41',
  'pledged': 692912.0,
  'state': 'successful',
  'backers': 2200.0,
  'country': 'US',
  'usd pledged': 692912.0,
  'usd_pledged_real': 692912.0},
 {'_id': ObjectId('5fccbfdc247348e3552645f1'),
  'ID': 918788305,
  'name': "DAN Cases A

In [22]:
cur=collection.find({"pledged":{ "$gte" :  200000}})
len(list(cur))

219

## Question 6 

collection.find({"name" : {"$regex": ".*Sport.*"}}).count(True)


In [23]:
cur=collection.find({"name" : {"$regex": ".*Sport.*"}})
len(list(cur))

76